# (공부) sequence_classification

신록예찬  
2024-08-28

In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [ ]:
load_dataset??

In [ ]:
imdb['train']

In [ ]:
imdb["test"][0]

> 리뷰 작성자는 자신이 공상과학을 좋아하지만, 이 시리즈는 매우
> 실망스러웠다고 말합니다. 저예산과 미흡한 캐릭터 개발, 어색한 대사,
> 저질 CG 등이 문제라고 언급하며, 시리즈가 지나치게 진지해지면서 재미와
> 활력을 잃었다고 평가합니다. 또한, 유명한 프로듀서의 이름을 남용해
> 시청자들의 관심을 끌려는 시도를 비판하며, 주인공을 죽였다가 다른
> 배우로 되살리는 전개가 어리석다고 평가합니다.

In [ ]:
imdb["test"][1]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
set(dir(tokenizer)) & {'__call__'}

In [ ]:
# 토큰화할 문장
text = "Hugging Face is creating a tool that democratizes AI."

# 텍스트를 토큰으로 변환
tokens = tokenizer.tokenize(text)
print(tokens)


In [ ]:
# 텍스트를 인풋 형태로 변환한 후, 인덱스 추출
text = "Hugging Face is creating a tool that democratizes AI."
inputs = tokenizer(text,truncation=True)
inputs

In [ ]:
# 텍스트를 인풋 형태로 변환한 후, 인덱스 추출
text = "Hugging Hugging Hugging"
inputs = tokenizer(text,truncation=True)
inputs

In [ ]:
# 텍스트를 인풋 형태로 변환한 후, 인덱스 추출
text = "Face Face Face face face face"
inputs = tokenizer(text,truncation=True)
inputs

In [ ]:
# 텍스트를 인풋 형태로 변환한 후, 인덱스 추출
text = "is is is"
inputs = tokenizer(text,truncation=True)
inputs

In [ ]:
set(dir(tokenizer)) & {'__call__'}

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(text)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]